In [ ]:
%run "./Setup.ipynb"

In [ ]:
inputPath = "E:\\PySpark\\data\\users.json"

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df1 = spark.read.json(inputPath)
df1.show()

In [ ]:
df1.createOrReplaceTempView("users")

In [ ]:
spark.catalog.listTables()

In [ ]:
qry = """select age, count(1) as count
        from users
        where age is not null
        group by age
        order by count
        limit 5"""

df2 = spark.sql(qry)

df2.show()

%md
#### GlobalTempViews

In [ ]:
df1.createOrReplaceGlobalTempView("gusers")

In [ ]:
spark.catalog.listTables("global_temp")

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
qry2 = """select age, count(1) as count
        from global_temp.gusers
        where age is not null
        group by age
        order by count
        limit 5"""

df3 = spark.sql(qry2)

df3.show()

#### Create a new session

In [ ]:
spark2 = spark.newSession()

In [ ]:
spark2.catalog.listTables()

In [ ]:
spark.catalog.listTables()

In [ ]:
print(qry2)

In [ ]:
df3 = spark2.sql(qry2)

df3.show()

#### Saving to tables

In [ ]:
df2 = df1.where("phone is not null") \
        .select("userid", "name", "gender", "age", "phone")
    
df2.show()

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
spark.conf.get("spark.sql.warehouse.dir")

In [ ]:
df2.write.format("json").saveAsTable("users_json")

In [22]:
spark.catalog.listTables()

[Table(name='users_csv', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='users_csv_ext', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='users_csv_partitoned', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='users_json', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [ ]:
spark.sql("select * from users_json where age > 20").show()

In [ ]:
df2.write.format("csv").option("header", "true").saveAsTable("users_csv")

In [25]:
spark.sql("select * from users_csv where age > 20").show()

+------+------+------+---+----------+
|userid|  name|gender|age|     phone|
+------+------+------+---+----------+
|     1| Satya|  Male| 25|8501099876|
|     4|Sandra|Female| 28|8508899001|
|     7| Steve|  Male| 28|8501085009|
|     8|Smriti|Female| 55|9246655498|
|     9| Veena|Female| 36|8508888888|
|    12| Raghu|  Male| 28|8501082222|
|    13| Ramya|Female| 55|9246654658|
|    14|Hasina|Female| 31|8345213235|
|    17|Mukesh|  Male| 31|8597638421|
+------+------+------+---+----------+



In [ ]:
df2 \
.write \
.format("csv") \
.option("header", "true") \
.option("path", "E:\\PySpark\\external\\users_csv_ext") \
.saveAsTable("users_csv_ext")

In [ ]:
df2 \
.write \
.format("csv") \
.option("header", "true") \
.partitionBy("gender") \
.saveAsTable("users_csv_partitoned")

In [24]:
spark.sql("SELECT * FROM users_csv_partitoned").show()

+------+-------+----+----------+------+
|userid|   name| age|     phone|gender|
+------+-------+----+----------+------+
|     1|  Satya|  25|8501099876|  Male|
|     2|   null|null|5676599876|  Male|
|     6| Sundar|  13|8522233456|  Male|
|     7|  Steve|  28|8501085009|  Male|
|    11| Samuel|  13|8522235624|  Male|
|    12|  Raghu|  28|8501082222|  Male|
|    16|Murugan|  13|8522209563|  Male|
|    17| Mukesh|  31|8597638421|  Male|
|     4| Sandra|  28|8508899001|Female|
|     8| Smriti|  55|9246655498|Female|
|     9|  Veena|  36|8508888888|Female|
|    13|  Ramya|  55|9246654658|Female|
|    14| Hasina|  31|8345213235|Female|
+------+-------+----+----------+------+

